# **LLM BRAIN**

In [42]:
from dotenv import load_dotenv
import os
import json
from langchain_core.prompts import ChatPromptTemplate
from functions import (
    init_gemini_client, 
    generate_trending_topic, 
    generate_full_pin_concept, 
    PinterestPinStrategy, 
    log_to_daily_jsonl
)
from prompts import get_system_prompt

load_dotenv()

if os.environ.get('GEMINI_API_KEY'):
    print("API KEY LOADED SUCCESSFULLY")
else:
    raise AttributeError("API KEY NOT FOUND")

API KEY LOADED SUCCESSFULLY


# **EXECUTION**

In [43]:
client_creative = init_gemini_client(temperature=0.7)

print("Generating Trending Topic...")
USER_PROMPT = generate_trending_topic(client_creative)
print(f"Topic: {USER_PROMPT}")

print("Expanding Concept...")
FULL_TOPIC = generate_full_pin_concept(client_creative, USER_PROMPT)

model_structured = init_gemini_client(temperature=0, response_mime_type="application/json")
structured_llm = model_structured.with_structured_output(PinterestPinStrategy)

prompt_template = ChatPromptTemplate.from_messages([
    ("system", get_system_prompt()),
    ("human", "{topic}")
])
chain = prompt_template | structured_llm

try:
    print("Generating Strategy JSON...")
    result = chain.invoke({'topic': FULL_TOPIC})

    with open('current_strategy.json', 'w') as f:
        if hasattr(result, 'model_dump'):
             json.dump(result.model_dump(), f)
        else:
             json.dump(result.dict(), f)
             
    log_file = log_to_daily_jsonl(result, folder="logs")
    print(f"✅ Success! Strategy saved to {log_file}")
    
except Exception as e:
    print(f"❌ Error: {e}")

Generating Trending Topic...
Topic: Cozy Neutral Home Decor
Expanding Concept...
Generating Strategy JSON...
✅ Success! Strategy saved to logs\2026-02-04_pinterest.jsonl
